This is the aumentation of 2 datasets: one with only diameter output (7 rows) and one with only emission output (7 rows). The moddel using Gradient Boosting will fill the absorbance column in each dataset.

In [79]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import joblib
import sklearn
# Import Scikit-Learn library for decision tree models
import sklearn 

from sklearn import datasets, linear_model, metrics, tree
from sklearn.utils import resample

from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor

from sklearn.linear_model import ElasticNet, Lasso,  RidgeCV

from sklearn.neighbors import KNeighborsRegressor

        
# Dataset splitting        
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_validate, RandomizedSearchCV

# Errors
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score

from matplotlib import pyplot as plt

from tqdm import tqdm

import os
import joblib

In [80]:
df = pd.read_csv("hao_dataset_scaled.csv")
df

,Unnamed: 0,in_amount_mmol,p_amount_mmol,sol_amount_ml,TOP_amount_mmol,acid_amount_mmol,amine_amount_mmol,thiol_amount_mmol,zinc_amount_mmol,other_amount_mmol,...,x8_dioctyl ether,x8_oleic acid,x8_tetrabutylammonium myristate,x8_trioctylamine,x8_trioctylphosphine oxide,x8_water,x8_zinc iodide,diameter_nm,abs_nm,emission_nm
0,0,-0.587378,-0.622362,-0.198864,-0.286002,-0.420577,-0.596494,0.378054,-0.498460,-0.254494,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,436,481
1,1,7.946556,6.875236,2.139673,7.645704,-0.420577,-0.596494,-0.185590,-0.581011,-0.254494,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,587,None
2,2,7.946556,7.859769,4.366852,-0.286002,-0.420577,-0.596494,-0.185590,-0.581011,-0.254494,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,465,550
3,3,0.102233,-0.584495,-0.365902,-0.286002,-0.305227,-0.552900,-0.185590,-0.581011,-0.254494,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,523,None
4,4,0.102233,-0.584495,-0.365902,-0.286002,-0.305227,-0.552900,-0.185590,-0.581011,-0.254494,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.1,584,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,241,1.334912,1.573904,-0.644300,4.056869,-0.420577,-0.596494,-0.185590,-0.581011,0.653295,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,729,775
242,242,2.946878,0.869585,-0.644300,2.093509,-0.420577,-0.596494,-0.185590,-0.581011,9.721213,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,None,599,612
243,243,1.274571,0.816571,-0.644300,2.886680,-0.420577,-0.596494,-0.185590,-0.581011,0.490691,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5,633,653
244,244,-0.587378,-0.622362,-0.644300,1.696924,-0.420577,-0.596494,-0.185590,1.070009,-0.254494,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,423,485


In [11]:
# Input for ML models

input_col = ['in_amount_mmol',
             'p_amount_mmol',
             'sol_amount_ml',
             'TOP_amount_mmol',
             'acid_amount_mmol',
             'amine_amount_mmol',
             'thiol_amount_mmol',
             'zinc_amount_mmol',
             'other_amount_mmol',
             'total_volume_ml',
             'temp_c',
             'time_min',
             'x0_chloroindium oxalate',
             'x0_indium acetate',
             'x0_indium bromide',
             'x0_indium chloride',
             'x0_indium iodide',
             'x0_indium laurate',
             'x0_indium myristate',
             'x0_indium oxalate',
             'x0_indium paltimate',
             'x0_indium stearate',
             'x0_indium trifluoroacetate',
             'x0_indium tris(N,N-diisopropylacetamidinato)',
             'x1_bis(trimethylsilyl)phosphine',
             'x1_phosphine gas',
             'x1_phosphorus trichloride',
             'x1_sodium phosphide',
             'x1_tris(diethylamino)phosphine',
             'x1_tris(dimethylamino)phosphine',
             'x1_tris(trimethylgermyl)phosphine',
             'x1_tris(trimethylsilyl)phosphine',
             'x1_white phosphorus',
             'x2_4-ethylpyridine',
             'x2_None',
             'x2_dimethylformamide',
             'x2_mesitylene',
             'x2_octadecene',
             'x2_toluene',
             'x3_None',
             'x3_trioctylphosphine',
             'x4_None',
             'x4_lauric acid',
             'x4_myristic acid',
             'x4_oleic acid',
             'x4_palmitic acid',
             'x4_stearic acid',
             'x5_None',
             'x5_dioctylamine',
             'x5_dodecylamine',
             'x5_hexadecylamine',
             'x5_octylamine',
             'x5_oleylamine',
             'x6_None',
             'x6_dodecanethiol',
             'x7_None',
             'x7_zinc acetate',
             'x7_zinc bromide',
             'x7_zinc chloride',
             'x7_zinc iodide',
             'x7_zinc octanoate',
             'x7_zinc oleate',
             'x7_zinc stearate',
             'x7_zinc undecylenate',
             'x8_None',
             'x8_acetic acid',
             'x8_copper bromide',
             'x8_dioctyl ether',
             'x8_oleic acid',
             'x8_tetrabutylammonium myristate',
             'x8_trioctylamine',
             'x8_trioctylphosphine oxide',
             'x8_water',
             'x8_zinc iodide'
            ]

In [38]:
# Load ML model for predicting absorbance

loaded_rf_ab = joblib.load('model_gradient_boosting_abs.joblib')

# Replace 'None' entries in 'abs_nm' column by predicted values.

a = 0
for index, row in df.iterrows():
    if row['abs_nm'] == 'None':
        X = df.loc[index, input_col].to_numpy()
        df.loc[index, 'abs_nm'] = loaded_rf_ab.predict(X.reshape(1, -1))[0]
        a += 1

        
# Save the dataset where all 'None' values in 'Absorbane max (nm)' column are replaced.   
df.to_csv('first_augmentation.csv')
df_first_aug = pd.read_csv('first_augmentation.csv')

In [18]:
#Saves the row indexes to drop for absorbance modelling into a list
total_row_num = len(df_first_aug)
drop_list_emi =[]
for row_i in range(total_row_num):
    if df_first_aug['emission_nm'].values[row_i] == 'None':
        drop_list_emi.append(row_i)
    
len(drop_list_emi)

#Drops the appropriate rows
df_emission_scaled_encoded = df_first_aug.drop(drop_list_emi)
#Saves the data for absorbance modelling to CSV
df_emission_scaled_encoded.to_csv('hao_all_emi_scaled.csv')

In [110]:
input_col_2 = ['in_amount_mmol',
             'p_amount_mmol',
             'sol_amount_ml',
             'TOP_amount_mmol',
             'acid_amount_mmol',
             'amine_amount_mmol',
             'thiol_amount_mmol',
             'zinc_amount_mmol',
             'other_amount_mmol',
             'total_volume_ml',
             'temp_c',
             'time_min',
             'x0_chloroindium oxalate',
             'x0_indium acetate',
             'x0_indium bromide',
             'x0_indium chloride',
             'x0_indium iodide',
             'x0_indium laurate',
             'x0_indium myristate',
             'x0_indium oxalate',
             'x0_indium paltimate',
             'x0_indium stearate',
             'x0_indium trifluoroacetate',
             'x0_indium tris(N,N-diisopropylacetamidinato)',
             'x1_bis(trimethylsilyl)phosphine',
             'x1_phosphine gas',
             'x1_phosphorus trichloride',
             'x1_sodium phosphide',
             'x1_tris(diethylamino)phosphine',
             'x1_tris(dimethylamino)phosphine',
             'x1_tris(trimethylgermyl)phosphine',
             'x1_tris(trimethylsilyl)phosphine',
             'x1_white phosphorus',
             'x2_4-ethylpyridine',
             'x2_None',
             'x2_dimethylformamide',
             'x2_mesitylene',
             'x2_octadecene',
             'x2_toluene',
             'x3_None',
             'x3_trioctylphosphine',
             'x4_None',
             'x4_lauric acid',
             'x4_myristic acid',
             'x4_oleic acid',
             'x4_palmitic acid',
             'x4_stearic acid',
             'x5_None',
             'x5_dioctylamine',
             'x5_dodecylamine',
             'x5_hexadecylamine',
             'x5_octylamine',
             'x5_oleylamine',
             'x6_None',
             'x6_dodecanethiol',
             'x7_None',
             'x7_zinc acetate',
             'x7_zinc bromide',
             'x7_zinc chloride',
             'x7_zinc iodide',
             'x7_zinc octanoate',
             'x7_zinc oleate',
             'x7_zinc stearate',
             'x7_zinc undecylenate',
             'x8_None',
             'x8_acetic acid',
             'x8_copper bromide',
             'x8_dioctyl ether',
             'x8_oleic acid',
             'x8_tetrabutylammonium myristate',
             'x8_trioctylamine',
             'x8_trioctylphosphine oxide',
             'x8_water',
             'x8_zinc iodide',
               'abs_nm'
            ]

In [85]:
# Now, the dataset only has 'None' values in the 'PL max (nm)' column

df_pl = pd.read_csv('first_augmentation.csv') 

# Load ML model for predicting PL

loaded_rf_pl = joblib.load('model_bagging_emission.joblib')


# # Replace 'None' entries in 'PL max (nm)' column by predicted values.

a = 0
for index, row in df_pl.iterrows():
    if row['emission_nm'] == 'None':
        X = df_pl.loc[index, input_col_2].to_numpy()
        df_pl.loc[index, 'emission_nm'] = loaded_rf_pl.predict(X.reshape(1, -1))[0]
        a += 1
# Y = X.reshape(1,-1)[0]
# # # Save the dataset where all 'None' values are replaced.
# # # Final augmented dataset.
# # # Ready to use for other analysis.
# Y
df_pl.to_csv('second_augmented.csv')

In [111]:
print(input_col_2)

['in_amount_mmol', 'p_amount_mmol', 'sol_amount_ml', 'TOP_amount_mmol', 'acid_amount_mmol', 'amine_amount_mmol', 'thiol_amount_mmol', 'zinc_amount_mmol', 'other_amount_mmol', 'total_volume_ml', 'temp_c', 'time_min', 'x0_chloroindium oxalate', 'x0_indium acetate', 'x0_indium bromide', 'x0_indium chloride', 'x0_indium iodide', 'x0_indium laurate', 'x0_indium myristate', 'x0_indium oxalate', 'x0_indium paltimate', 'x0_indium stearate', 'x0_indium trifluoroacetate', 'x0_indium tris(N,N-diisopropylacetamidinato)', 'x1_bis(trimethylsilyl)phosphine', 'x1_phosphine gas', 'x1_phosphorus trichloride', 'x1_sodium phosphide', 'x1_tris(diethylamino)phosphine', 'x1_tris(dimethylamino)phosphine', 'x1_tris(trimethylgermyl)phosphine', 'x1_tris(trimethylsilyl)phosphine', 'x1_white phosphorus', 'x2_4-ethylpyridine', 'x2_None', 'x2_dimethylformamide', 'x2_mesitylene', 'x2_octadecene', 'x2_toluene', 'x3_None', 'x3_trioctylphosphine', 'x4_None', 'x4_lauric acid', 'x4_myristic acid', 'x4_oleic acid', 'x4_pal

In [112]:
input_col_2.append('emission_nm')
input_col_3 = input_col_2.copy()


In [ ]:
print(input_col_3)
input_col_2.remove('emission_nm')

In [87]:
df_second_aug = pd.read_csv('second_augmentation.csv')
#Saves the row indexes to drop for diameter modelling into a list
total_row_num = len(df_second_aug)
drop_list_dia =[]
for row_i in range(total_row_num):
    if df_second_aug['diameter_nm'].values[row_i] == 'None':
        drop_list_dia.append(row_i)
    
len(drop_list_dia)

#Drops the appropriate rows
df_dia_scaled_encoded = df_second_aug.drop(drop_list_dia)
#Saves the data for absorbance modelling to CSV
df_dia_scaled_encoded.to_csv('hao_all_dia_scaled.csv')

In [117]:
# Load ML model for predicting diameters

df_dia = pd.read_csv('second_augmentation.csv') 

loaded_rf_dia = joblib.load('model_decision_tree_diameter.joblib')

# Replace 'None' entries in 'diameter_nm' column by predicted values.

a = 0
for index, row in df_dia.iterrows():
    if row['diameter_nm'] == 'None':
        X = df_dia.loc[index, input_col_3].to_numpy()
        df_dia.loc[index, 'diameter_nm'] = loaded_rf_dia.predict(X.reshape(1, -1))[0]
        a += 1

        
# Save the dataset where all 'None' values in 'Absorbane max (nm)' column are replaced.   
df_dia.to_csv('hao_augmented_dataset.csv')